# Import Libraries

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pandas as pd
import logging

In [2]:
data=pd.read_csv('Preprocessed_data.csv')

#Dividing target variable from the main dataset

X = data.iloc[: , 0:-1]
Y = data.iloc[: , -1] 

In [3]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1)

# Encoding categorical data
## Encoding the Independent Variable

In [4]:
ordinal_encoder = OrdinalEncoder()
X_train_cat_encoded = pd.DataFrame(ordinal_encoder.fit_transform(X_train.select_dtypes(exclude='number')))
X_train_cat_encoded.columns = X_train.select_dtypes(exclude='number').columns

X_test_cat_encoded = pd.DataFrame(ordinal_encoder.transform(X_test.select_dtypes(exclude='number')))
X_test_cat_encoded.columns = X_test.select_dtypes(exclude='number').columns

## Encoding the Independent Variable

In [5]:
label_encoder = LabelEncoder()
Y_train_cat_encoded= pd.DataFrame(label_encoder.fit_transform(Y_train))
Y_test_cat_encoded = pd.DataFrame(label_encoder.transform(Y_test))

## Standardization

In [6]:
sc = StandardScaler()
X_train_sc=pd.DataFrame(sc.fit_transform(X_train.select_dtypes(exclude='O')))
X_test_sc=pd.DataFrame(sc.transform(X_test.select_dtypes(exclude='O')))

X_train_sc.columns=X_train.select_dtypes(exclude='O').columns
X_test_sc.columns=X_test.select_dtypes(exclude='O').columns

## Combining data

In [7]:
X_train_final=pd.concat([X_train_sc,X_train_cat_encoded],axis=1)
X_test_final=pd.concat([X_test_sc,X_test_cat_encoded],axis=1)

## Handling imbalanced Dataset
### Since the dataset is small, will use over-sampling: SMOTE technique to balance the data

In [8]:
X_train_resample,Y_train_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_train_final,Y_train_cat_encoded)
X_test_resample,Y_test_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_test_final,Y_test_cat_encoded)

X_train_resample.shape,X_test_resample.shape,Y_train_resample.shape,Y_test_resample.shape

((7852, 28), (1476, 28), (7852, 1), (1476, 1))

## Feature Selection

In [9]:
print('Training dataset shape:', X_train_resample.shape, Y_train_resample.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample.shape)

Y_train_resample_flat = Y_train_resample.to_numpy().ravel()
Y_test_resample_flat = Y_test_resample.to_numpy().ravel()

print('Training dataset shape:', X_train_resample.shape, Y_train_resample_flat.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample_flat.shape)

Training dataset shape: (7852, 28) (7852, 1)
Testing dataset shape: (1476, 28) (1476, 1)
Training dataset shape: (7852, 28) (7852,)
Testing dataset shape: (1476, 28) (1476,)


### Forward selection approach

In [10]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
forward_fs = sfs(rf , k_features=10,forward=True,floating=False,verbose=2,scoring='accuracy',cv=5)
forward_fs = forward_fs.fit(X_train_resample, Y_train_resample_flat)


[2025-04-24 12:59:57] Features: 1/10 -- score: 0.7903702051920355
[2025-04-24 13:00:56] Features: 2/10 -- score: 0.9680334242865308
[2025-04-24 13:01:55] Features: 3/10 -- score: 0.9890475862264694
[2025-04-24 13:03:05] Features: 4/10 -- score: 0.9950328201843119
[2025-04-24 13:04:03] Features: 5/10 -- score: 0.9974528779997325
[2025-04-24 13:04:58] Features: 6/10 -- score: 0.9982168037721927
[2025-04-24 13:05:53] Features: 7/10 -- score: 0.9982168037721927
[2025-04-24 13:06:41] Features: 8/10 -- score: 0.9980894152371608
[2025-04-24 13:07:39] Features: 9/10 -- score: 0.9973254083771543
[2025-04-24 13:08:27] Features: 10/10 -- score: 0.9974526347370938

In [11]:
# Create the Handler for logging records/messages to a file
file_handler = logging.FileHandler("log_file.log")

In [12]:
#set the format of the log records and the logging level to DEBUG
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG)

In [13]:
# a function  to create and save logs in the log files
def log(path, file):
    """[Create a log file to record the experiment's logs]
    
    Arguments:
        path {string} -- path to the directory
        file {string} -- file name
    
    Returns:
        [obj] -- [logger that record logs]
    """

    # check if the file exist
    log_file = os.path.join(path, file)

    if not os.path.isfile(log_file):
        open(log_file, "w+").close()

    console_logging_format = "%(levelname)s %(message)s"
    file_logging_format = "%(levelname)s: %(asctime)s: %(message)s"

    # configure logger
    logging.basicConfig(level=logging.INFO, format=console_logging_format)
    logger = logging.getLogger()
    
    # create a file handler for output file
    handler = logging.FileHandler(log_file)

    # set the logging level for log file
    handler.setLevel(logging.INFO)
    
    # create a logging format
    formatter = logging.Formatter(file_logging_format)
    handler.setFormatter(formatter)

    # add the handlers to the logger
    logger.addHandler(handler)

    return logger

In [14]:
import os

In [15]:
logger = log(path=".",file="log_file.log")

In [16]:
logger.info("Feature Selection")

2025-04-24 13:08:45,679 INFO:Feature Selection


In [17]:
feat_names = list(forward_fs.k_feature_names_)
logger.info("Features {}".format(feat_names))
X_train_new=X_train_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured']]
X_test_new=X_test_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured']]

2025-04-24 13:08:46,823 INFO:Features ['age', 'TSH', 'TT4', 'FTI', 'on_thyroxine', 'pregnant', 'hypopituitary', 'psych', 'TT4_measured', 'referral_source']


In [18]:
#Fitting the Random Forest model
rf_model=rf.fit(X_train_new,Y_train_resample_flat)

logger.info("Traininig Random Forest Model")

#Checking the metrics of Random Forest
def print_Score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        pred=clf.predict(x_train)
        clf_report=pd.DataFrame(classification_report(y_train,pred,output_dict=True))
        print("Train Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_train,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("-----------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_train,pred)}\n")
        logger.info("Train Result:\n==============="+"\n"+
                    f"Accuracy Score:{accuracy_score(y_train,pred)*100:.2f}%""\n"+
        "---------------------------------"+"\n"+
        f"Classification Report:\n{clf_report}"+"\n"+
        "-----------------------------------"+"\n"+
        f"Confusion Matrix:\n{confusion_matrix(y_train,pred)}\n")
    elif train==False:
        pred=clf.predict(x_test)
        clf_report=pd.DataFrame(classification_report(y_test,pred,output_dict=True))
        print("Test Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("---------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_test,pred)}\n")
        
        logger.info("Test Result:\n==============="+"\n"+
        f"Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%"+"\n"+
        "---------------------------------"+"\n"+
        f"Classification Report:\n{clf_report}"+"\n"+
        "---------------------------------"+"\n"+
        f"Confusion Matrix:\n{confusion_matrix(y_test,pred)}\n")
        
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2025-04-24 13:08:49,295 INFO:Traininig Random Forest Model
2025-04-24 13:08:49,350 INFO:Train Result:
Accuracy Score:99.80%
---------------------------------
Classification Report:
                     0            1            2       3  accuracy  \
precision     0.993421     1.000000     0.998474     1.0  0.997962   
recall        1.000000     0.991849     1.000000     1.0  0.997962   
f1-score      0.996700     0.995908     0.999236     1.0  0.997962   
support    1963.000000  1963.000000  1963.000000  1963.0  0.997962   

             macro avg  weighted avg  
precision     0.997974      0.997974  
recall        0.997962      0.997962  
f1-score      0.997961      0.997961  
support    7852.000000   7852.000000  
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [  13 1947    3    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

C:\Users\Bharath\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarni

Train Result:
Accuracy Score:99.80%
---------------------------------
Classification Report:
                     0            1            2       3  accuracy  \
precision     0.993421     1.000000     0.998474     1.0  0.997962   
recall        1.000000     0.991849     1.000000     1.0  0.997962   
f1-score      0.996700     0.995908     0.999236     1.0  0.997962   
support    1963.000000  1963.000000  1963.000000  1963.0  0.997962   

             macro avg  weighted avg  
precision     0.997974      0.997974  
recall        0.997962      0.997962  
f1-score      0.997961      0.997961  
support    7852.000000   7852.000000  
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [  13 1947    3    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

Test Result:
Accuracy Score:97.22%
---------------------------------
Classification Report:
                    0           1           2    3  accuracy    macro avg  \
precision    0.924812    1.000000    1.00000

In [19]:
##Hyper parameter tuning
RF=RandomForestClassifier()
model=RF.fit(X_train_new,Y_train_resample_flat)

print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2025-04-24 13:08:52,080 INFO:Train Result:
Accuracy Score:100.00%
---------------------------------
Classification Report:
                0       1       2       3  accuracy  macro avg  weighted avg
precision     1.0     1.0     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0     1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0     1.0     1.0       1.0        1.0           1.0
support    1963.0  1963.0  1963.0  1963.0       1.0     7852.0        7852.0
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   0 1963    0    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

2025-04-24 13:08:52,112 INFO:Test Result:
Accuracy Score:98.37%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.953488    1.000000    1.000000   0.98374     0.984496   
recall       1.000000    0.995935    0.955285   0.98374     0.983740   
f1-scor

Train Result:
Accuracy Score:100.00%
---------------------------------
Classification Report:
                0       1       2       3  accuracy  macro avg  weighted avg
precision     1.0     1.0     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0     1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0     1.0     1.0       1.0        1.0           1.0
support    1963.0  1963.0  1963.0  1963.0       1.0     7852.0        7852.0
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   0 1963    0    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

Test Result:
Accuracy Score:98.37%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.953488    1.000000    1.000000   0.98374     0.984496   
recall       1.000000    0.995935    0.955285   0.98374     0.983740   
f1-score     0.976190    0.997963    0.977131   0.98374     0.983

In [20]:
import numpy as np

In [21]:
## Randomized Search CV

#No of trees in Random Forest
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
#No of features consider at every split
max_features=['auto','sqrt','log2']
#maximum no of levels in trees
max_depth=[int(x) for x in np.linspace(10,1000,10)]
#minimum no of samples required to split a node
min_samples_split=[1,3,4,5,7,9]
#minimum samples leafs required at each leaf node
min_sample_leafs=[1,2,4,6,8]

#create random gird
random_grid={'n_estimators':n_estimators,
'max_features':max_features,
'max_depth':max_depth,
'min_samples_split':min_samples_split,
'min_samples_leaf':min_sample_leafs,
'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [22]:
rcv=RandomizedSearchCV(estimator=RF,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,random_state=0,n_jobs=-1)

rcv.fit(X_train_new,Y_train_resample_flat)

logger.info("Use Randomized Search CV")

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\Bharath\AppData\Roaming\Python\Python313\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
111 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
31 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Bharath\AppData\Roaming\Python\Python313\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bharath\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\Bharath\AppData\Roami

In [23]:
rcv.best_estimator_
logger.info(rcv.best_estimator_)

2025-04-24 13:16:17,550 INFO:RandomForestClassifier(criterion='entropy', max_depth=560, min_samples_split=9,
                       n_estimators=400)


In [24]:
best_random_grid=rcv.best_estimator_
logger.info("Result with best estimetors")
print_Score(best_random_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2025-04-24 13:16:19,305 INFO:Result with best estimetors
2025-04-24 13:16:19,456 INFO:Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



In [25]:
param_grid = {
    'criterion': [rcv.best_params_['criterion']],
    'max_depth': [rcv.best_params_['max_depth']],
    'max_features': [rcv.best_params_['max_features']],
    'min_samples_leaf': [rcv.best_params_['min_samples_leaf'], 
                         rcv.best_params_['min_samples_leaf']+2, 
                         rcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rcv.best_params_['min_samples_split'] - 2,
                          rcv.best_params_['min_samples_split'] - 1,
                          rcv.best_params_['min_samples_split'], 
                          rcv.best_params_['min_samples_split'] +1,
                          rcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rcv.best_params_['n_estimators'] - 200, rcv.best_params_['n_estimators'] - 100, 
                     rcv.best_params_['n_estimators'], 
                     rcv.best_params_['n_estimators'] + 100, rcv.best_params_['n_estimators'] + 200]
}

In [26]:
print(param_grid)
logger.info(f"Parameter Grid: {param_grid}")

2025-04-24 13:16:21,509 INFO:Parameter Grid: {'criterion': ['entropy'], 'max_depth': [560], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [7, 8, 9, 10, 11], 'n_estimators': [200, 300, 400, 500, 600]}


{'criterion': ['entropy'], 'max_depth': [560], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [7, 8, 9, 10, 11], 'n_estimators': [200, 300, 400, 500, 600]}


In [27]:
grid_search=GridSearchCV(estimator=RF,param_grid=param_grid,cv=10,n_jobs=1,verbose=2)
grid_search.fit(X_train_new,Y_train_resample_flat)
logger.info("Grid Search CV: ")

Fitting 10 folds for each of 75 candidates, totalling 750 fits
[CV] END criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200; total time=   1.7s
[CV] END criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200; total time=   1.7s
[CV] END criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200; total time=   1.7s
[CV] END criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200; total time=   1.7s
[CV] END criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200; total time=   1.6s
[CV] END criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_split=7, n_estimators=200; total time=   1.7s
[CV] END criterion=entropy, max_depth=560, max_features=sqrt, min_samples_leaf=1, min_samples_spl

2025-04-24 14:00:19,713 INFO:Grid Search CV: 


In [28]:
best_grid=grid_search.best_estimator_
best_grid
logger.info(f"Best Grid: {best_grid}")

2025-04-24 14:00:19,747 INFO:Best Grid: RandomForestClassifier(criterion='entropy', max_depth=560, min_samples_split=7,
                       n_estimators=400)


In [29]:
logger.info("Result with Best Grid: ")
print_Score(best_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2025-04-24 14:20:54,030 INFO:Result with Best Grid: 
2025-04-24 14:20:54,170 INFO:Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



In [30]:
#Now, will convert our final model into pickle file

import pickle

pickle_out=open('Thyroid_model.pkl','wb')
pickle.dump(grid_search,pickle_out)
pickle_out.close()
logger.info("Random Forest Model with Grid Search Saved in Pickle")

2025-04-24 14:20:55,450 INFO:Random Forest Model with Grid Search Saved in Pickle
